# Insights - Total population by year

## Introduction

This process is dependent on upstream processes. See the "Prerequisites" section below.

The workflow defined herein is identified as workflow ID #90 in the the [Data Team Master Document List](https://morpc1.sharepoint.com/:x:/s/GISteam/EfC4j3HhohZCrSZzxJdyt5cBFEqVD7zHick8ZW0INqgCYA?e=0WhrAI). References to document list identifiers are denoted by a number in brackets, e.g. [90].

## Process outline

  1. Load input dataset
  2. Extract required population facts
  3. Transform population facts to comply with output schema
  4. Export output dataset
  5. Create resource file

## Prerequisites and usage notes

  - Outputs of one or more upstream workflows must be available at the indicated paths. Make sure that those outputs are up to date prior to running this script. 
  - This script includes several intentional RuntimeError instances that may be triggered to alert the user to conditions that may require their attention. If the script triggers one of these errors, review the error, verify that the condition is acceptable or resolve any issues, then proceed.

## Setup

### Import required packages

In [ ]:
import os
import shutil
import sys
import pandas as pd
import frictionless
import datetime
import matplotlib
from matplotlib import pyplot as plt
sys.path.append(os.path.normpath("../morpc-common"))
import morpc
import morpcCensus

### User-specified parameters

In [ ]:
MORPC_ESTIMATE_YEAR_RANGE = [2024, 2024]

MORPC_FORECAST_VINTAGE = 2023
MORPC_FORECAST_YEAR_RANGE = [2025, 2050]
MORPC_FORECAST_YEAR_INTERVAL = 5

DECENNIAL_YEAR_RANGE = [1980, 2020]
INTERCENSAL_YEAR_RANGE = [2000, 2019]

PEP_YEAR_RANGE = [INTERCENSAL_YEAR_RANGE[1]+1, MORPC_ESTIMATE_YEAR_RANGE[0]-1]

FORECAST_POP_THRESHOLD = 2000
INCLUSION_POP_THRESHOLD = 100

# When STALE_DATA_INTERRUPT == True, the script will produce a RuntimeError in certain situations where the input 
# data may be stale and updates might be required prior to running the script.  Otherwise, a warning will be generated 
# but script execution will continue.  Regardless of whether an error or warning occurs, be sure to verify the readiness 
# of all input data.
STALE_DATA_INTERRUPT = True

# You can change where the input data is sourced and archived by changing the following directory and file names.  
# This typically is not necessary and may break other scripts that depend on outputs from this one. Source data 
# will be copied to this location.  Input data will be deleted following successful completion of the script 
# unless PRESERVE_INPUT_DATA == True.
INPUT_DIR = "./input_data"

# You can change where the output data is stored by changing the following directory and file names.  This 
# typically is not necessary and may break other scripts that depend on outputs from this one.
OUTPUT_DIR = "./output_data"

### Static parameters

Create a map to convert human-readable source descriptions to shortened codes to save space.

In [ ]:
GEO_TYPE_LIST = ["COUNTY","PLACE","COUNTY-TOWNSHIP-REMAINDER"]

SOURCE_MAP = {
    "Census Intercensal Estimates":"CENINT",
    "Census Population Estimates Program":"CENPEP",
    "Mid-Ohio Regional Planning Commission":"MORPC"
}

SOURCE_MAP_REVERSED = {value: key for key, value in SOURCE_MAP.items()}

GEO_TYPE_LABELS = {
    "REGION15":"",
    "COUNTY":"",
    "COUNTY-TOWNSHIP-REMAINDER":" (unincorporated)",
    "PLACE":""
}

CHART_DIRNAME = "charts"

### Define inputs

The following datasets are required by this notebook. They will be retrieved from the specified location and temporarily stored in INPUT_DIR. They will be deleted following successful completion of the script unless PRESERVE_INPUT_DATA == True.

#### Create input data directory

Create input data directory if it doesn't exist.

In [ ]:
inputDir = os.path.normpath(INPUT_DIR)
if not os.path.exists(inputDir):
    os.makedirs(inputDir)

#### MORPC counties reference data [81]

Reference data for counties in the MORPC region will be loaded automatically as a morpc.countyLookup() object (see below).

#### MORPC combined population facts [286]

In [ ]:
COMBINED_POP_FACTS_RESOURCE_PATH = "../morpc-pop-collect/output_data/morpc-pop-collect.resource.yaml"
print("Resource file: {}".format(COMBINED_POP_FACTS_RESOURCE_PATH))

#### MORPC geography lookup table [375]

In [ ]:
GEOS_LOOKUP_RESOURCE_PATH = "../morpc-geos-collect/output_data/morpc-geos-lookup.resource.yaml"
print("Resource file: {}".format(GEOS_LOOKUP_RESOURCE_PATH))

#### MORPC member list [122]

In [ ]:
MEMBERS_DATA_PATH = "../morpc-lookup/Member_List.xlsx"
MEMBERS_SHEET = "Current Year Members"
MEMBERS_SCHEMA_PATH = "../morpc-lookup/Member_List_schema.json"
print("Data: {}, sheet '{}'".format(MEMBERS_DATA_PATH, MEMBERS_SHEET))
print("Schema: {}".format(MEMBERS_SCHEMA_PATH))

### Define outputs

#### Create output data directory

Create output data directory if it doesn't exist.

In [ ]:
outputDir = os.path.normpath(OUTPUT_DIR)
if not os.path.exists(outputDir):
    os.makedirs(outputDir)   

In [ ]:
chartDir = os.path.join(outputDir, CHART_DIRNAME)
if not os.path.exists(chartDir):
    os.makedirs(chartDir)    

#### Insights total population by year [287]

In [ ]:
INSIGHTS_POP_TABLE_FILENAME = "morpc-insights-pop-temporal.csv"
INSIGHTS_POP_TABLE_PATH = os.path.join(outputDir, INSIGHTS_POP_TABLE_FILENAME)
INSIGHTS_POP_TABLE_SCHEMA_PATH = INSIGHTS_POP_TABLE_PATH.replace(".csv",".schema.yaml")
INSIGHTS_POP_TABLE_RESOURCE_PATH = INSIGHTS_POP_TABLE_PATH.replace(".csv",".resource.yaml")
print("Data: {}".format(INSIGHTS_POP_TABLE_PATH))
print("Schema: {}".format(INSIGHTS_POP_TABLE_SCHEMA_PATH))
print("Resource file: {}".format(INSIGHTS_POP_TABLE_RESOURCE_PATH))

## Prepare input data

### Load county reference data

In [ ]:
countyLookup = morpc.countyLookup(scope="15-County Region")

In [ ]:
",".join(countyLookup.list_ids())

### Combined population facts

In [ ]:
(combinedPopRaw, combinedPopResource, combinedPopSchema) = morpc.frictionless_load_data(COMBINED_POP_FACTS_RESOURCE_PATH, archiveDir=inputDir, validate=True, verbose=True)

In [ ]:
combinedPop = combinedPopRaw.copy()

In [ ]:
combinedPop["GEO_TYPE"] = combinedPop["SUMLEVEL"].map(morpc.HIERARCHY_STRING_LOOKUP)

In [ ]:
combinedPop = combinedPop.drop(columns=["SUMLEVEL","LAST_UPDATED"]) 

In [ ]:
combinedPop = combinedPop.loc[combinedPop["GEO_TYPE"].isin(GEO_TYPE_LIST)]

In [ ]:
combinedPop.head()

### Geography lookup table

In [ ]:
(geoLookupRaw, geoLookupResource, geoLookupSchema) = morpc.frictionless_load_data(GEOS_LOOKUP_RESOURCE_PATH, archiveDir=inputDir, validate=True, verbose=True)

In [ ]:
geoLookup = geoLookupRaw.copy() \
    .set_index("GEOIDFQ")

In [ ]:
geoLookup.head()

### MORPC member list

The output data will include all communities for whom data has not been suppressed, however we will only show data for MORPC members in the platform.

Load the member table.

In [ ]:
membersRaw = pd.read_excel(MEMBERS_DATA_PATH, sheet_name=MEMBERS_SHEET)
membersRaw.head()

Load the schema.

In [ ]:
membersSchema = morpc.load_avro_schema(MEMBERS_SCHEMA_PATH)

Verify that the fields are all the expected types.

In [ ]:
members = morpc.cast_field_types(membersRaw, membersSchema)

Extract only the communities which are themselves a member.

In [ ]:
members = members.loc[members["Local Member"] == True].copy()

The records in the member table are all county parts.  For places we need to subsitute the GEOID for the full place rather than the county part.

In [ ]:
members["PLACEFP"] = members["GEOID"].apply(lambda x:x[11:16])
members["COUSUBFP"] = members["GEOID"].apply(lambda x:x[14:19])
members["COUNTYID"] = members["County"].map(morpc.CONST_COUNTY_NAME_TO_ID)
members["GEOIDFQ"] = None
temp = members.loc[members["GovType"] == "Township"].copy()
temp["GEOIDFQ"] = temp["GEOID"]
members.update(temp)
temp = members.loc[members["GovType"] != "Township"].copy()
temp["GEOIDFQ"] = "1600000US39" + temp["PLACEFP"]
members.update(temp)

Now extract just the list of member GEOIDs.  The steps above likely produced duplicate records for places, so extract only the unique GEOIDs.

In [ ]:
memberList = list(members["GEOIDFQ"].unique())

We also need to append the list of counties.  We will include data for all counties regardless of membership status.

In [ ]:
memberList += ["0500000US{}".format(morpc.CONST_COUNTY_NAME_TO_ID[x]) for x in morpc.CONST_REGIONS['REGION15']]

Finally we need to append the ID for the MORPC region.

In [ ]:
memberList.append("M010000US001")

## Transform data

#### Load output schema

In [ ]:
insightsPopSchema = morpc.frictionless_load_schema(INSIGHTS_POP_TABLE_SCHEMA_PATH)
insightsPopSchema

### Create list to collect extracted data

In [ ]:
extractedData = []

### Extract decennial census counts (NOT IMPLEMENTED)

### Extract intercensal estimates

Create list of years from user-specified range.

In [ ]:
intercensalRange = list(range(INTERCENSAL_YEAR_RANGE[0], INTERCENSAL_YEAR_RANGE[1]+1))
print("Including intercensal estimates for years: {}".format(", ".join([str(x) for x in intercensalRange])))

Extract intercensal data from combined table.

In [ ]:
intercensal = combinedPop.loc[combinedPop["SOURCE"] == "CENINT"].copy()

Reference period and vintage period were stored as strings, but for intercensal estimates we can treat them as integers. Convert them now.

In [ ]:
intercensal = intercensal.astype({
    "REFERENCE_PERIOD":"int",
    "VINTAGE_PERIOD":"int"
})

Verify that data is available for the specified years.

In [ ]:
if(not set(intercensalRange).issubset(set(intercensal["REFERENCE_PERIOD"]))):
    print("ERROR | Set of intercensal years for which data is available does not match set derived from specified range.")
    print("Specified range: {}".format(INTERCENSAL_YEAR_RANGE))
    print("Specified set: {}".format(set(intercensalRange)))
    print("Available set: {}".format(set(intercensal["REFERENCE_PERIOD"])))
    raise RuntimeError
else:
    print("INFO | Intercensal data is available for all years in specified range.")

Extract only the estimates for the specified years.

In [ ]:
intercensal = morpc.extract_vintage(intercensal, refPeriods=intercensalRange)

Verify that all reference periods are present and that there is only a single vintage for each reference period.

In [ ]:
for year in intercensal["REFERENCE_PERIOD"].unique():
    temp = intercensal.loc[intercensal["REFERENCE_PERIOD"] == year]
    print("{0}: {1}".format(year, ",".join(temp["VINTAGE_PERIOD"].unique().astype("str"))))

Construct DATE field from reference period and reference period frequency.

In [ ]:
if(intercensal["REFERENCE_PERIOD_FREQ"].unique().shape[0] == 1):
    freq = intercensal["REFERENCE_PERIOD_FREQ"].iat[0]
    print("INFO | Detected reference period frequency {}".format(freq))
else:
    print("ERROR | Multiple reference period frequencies are not supported.")
    raise RuntimeError

try:
    # Hopefully this works properly with newer versions of pandas, but it has not been tested
    periodIndex = pd.PeriodIndex(intercensal["REFERENCE_PERIOD"]+1, freq=freq)
except:
    # This works with older versions of pandas. Note the +1 offset. For whatever reason, pandas converts 2000 Y-JUN (for example)
    # to a timestamp of 1999-07-01 so we add one year to the reference period so the timestamp becomes 2000-07-01.
    print("WARNING | Error occurred when attempting to create period index using 'Y-' format. Trying legacy 'A-' format.")
    periodIndex = pd.PeriodIndex(intercensal["REFERENCE_PERIOD"]+1, freq=freq.replace("Y-","A-"))
intercensal["DATE"] = periodIndex.to_timestamp()

Show the data.

In [ ]:
intercensal.head()

In [ ]:
extractedData.append(intercensal)

### Extract PEP estimates

Create list of years from user-specified range.

In [ ]:
pepRange = list(range(PEP_YEAR_RANGE[0], PEP_YEAR_RANGE[1]+1))
print("Including PEP estimates for years: {}".format(", ".join([str(x) for x in pepRange])))

Extract PEP estimates from combined table.

In [ ]:
pep = combinedPop.loc[combinedPop["SOURCE"] == "CENPEP"].copy()

Reference period and vintage period were stored as strings, but for Census PEP estimates we can treat them as integers. Convert them now.

In [ ]:
pep = pep.astype({
    "REFERENCE_PERIOD":"int",
    "VINTAGE_PERIOD":"int"
})

Verify that data is available for the specified years.

In [ ]:
if(not set(pepRange).issubset(set(pep["REFERENCE_PERIOD"]))):
    print("ERROR | Set of Census PEP years for which data is available does not match set derived from specified range.")
    print("Specified range: {}".format(PEP_YEAR_RANGE))
    print("Specified set: {}".format(set(pepRange)))
    print("Available set: {}".format(set(pep["REFERENCE_PERIOD"])))
    raise RuntimeError
else:
    print("INFO | PEP data is available for all years in specified range.")

Extract only the estimates for the specified years.

In [ ]:
pep = morpc.extract_vintage(pep, refPeriods=pepRange)

Verify that all reference periods are present and that there is only a single vintage for each reference period.

In [ ]:
for year in pep["REFERENCE_PERIOD"].unique():
    temp = pep.loc[pep["REFERENCE_PERIOD"] == year]
    print("{0}: {1}".format(year, ",".join(temp["VINTAGE_PERIOD"].unique().astype("str"))))

Construct DATE field from reference period and reference period frequency.

In [ ]:
if(pep["REFERENCE_PERIOD_FREQ"].unique().shape[0] == 1):
    freq = pep["REFERENCE_PERIOD_FREQ"].iat[0]
    print("INFO | Detected reference period frequency {}".format(freq))
else:
    print("ERROR | Multiple reference period frequencies are not supported.")
    raise RuntimeError

try:
    # Hopefully this works properly with newer versions of pandas, but it has not been tested
    periodIndex = pd.PeriodIndex(pep["REFERENCE_PERIOD"]+1, freq=freq)
except:
    # This works with older versions of pandas. Note the +1 offset. For whatever reason, pandas converts 2000 Y-JUN (for example)
    # to a timestamp of 1999-07-01 so we add one year to the reference period so the timestamp becomes 2000-07-01.
    print("WARNING | Error occurred when attempting to create period index using 'Y-' format. Trying legacy 'A-' format.")
    periodIndex = pd.PeriodIndex(pep["REFERENCE_PERIOD"]+1, freq=freq.replace("Y-","A-"))
pep["DATE"] = periodIndex.to_timestamp()

Show the data.

In [ ]:
pep.head()

In [ ]:
extractedData.append(pep)

### Extract MORPC county estimates

Because county estimates and sub-county estimates are generated (and regenerated) at different times, it is necessary to process each separately. Start with county estimates.

Create list of years from user-specified range.

In [ ]:
morpcEstimatesRange = list(range(MORPC_ESTIMATE_YEAR_RANGE[0], MORPC_ESTIMATE_YEAR_RANGE[1]+1))
print("Including MORPC estimates for years: {}".format(", ".join([str(x) for x in morpcEstimatesRange])))

Extract MORPC county estimates from combined table.

In [ ]:
morpcCountyEstimates = combinedPop.loc[
    (combinedPop["SOURCE"] == "MORPC") & 
    (combinedPop["VALUE_TYPE"] == "ESTIMATE") &
    (combinedPop["GEO_TYPE"] == "COUNTY")
].copy()

Reference period and vintage period were stored as strings, but for MORPC estimates we can treat them as integers. Convert them now.

In [ ]:
morpcCountyEstimates = morpcCountyEstimates.astype({
    "REFERENCE_PERIOD":"int",
    "VINTAGE_PERIOD":"int"
})

Verify that data is available for the specified years.

In [ ]:
if(not set(morpcEstimatesRange).issubset(set(morpcCountyEstimates["REFERENCE_PERIOD"]))):
    print("ERROR | Set of MORPC estimate years for which data is available does not match set derived from specified range.")
    print("Specified range: {}".format(MORPC_ESTIMATE_YEAR_RANGE))
    print("Specified set: {}".format(set(morpcEstimatesRange)))
    print("Available set: {}".format(set(morpcCountyEstimates["REFERENCE_PERIOD"])))
    raise RuntimeError
else:
    print("INFO | MORPC county estimates data is available for all years in specified range.")

Extract only the estimates for the specified years.

In [ ]:
morpcCountyEstimates = morpc.extract_vintage(morpcCountyEstimates, refPeriods=morpcEstimatesRange)

Verify that all reference periods are present and that there is only a single vintage for each reference period.

In [ ]:
for year in morpcCountyEstimates["REFERENCE_PERIOD"].unique():
    temp = morpcCountyEstimates.loc[morpcCountyEstimates["REFERENCE_PERIOD"] == year]
    print("{0}: {1}".format(year, ",".join(temp["VINTAGE_PERIOD"].unique().astype("str"))))

Construct DATE field from reference period and reference period frequency.

In [ ]:
if(morpcCountyEstimates["REFERENCE_PERIOD_FREQ"].unique().shape[0] == 1):
    freq = morpcCountyEstimates["REFERENCE_PERIOD_FREQ"].iat[0]
    print("INFO | Detected reference period frequency {}".format(freq))
else:
    print("ERROR | Multiple reference period frequencies are not supported.")
    raise RuntimeError

try:
    # Hopefully this works properly with newer versions of pandas, but it has not been tested
    periodIndex = pd.PeriodIndex(morpcCountyEstimates["REFERENCE_PERIOD"], freq=freq)
except:
    # This works with older versions of pandas.
    print("WARNING | Error occurred when attempting to create period index using 'Y-' format. Trying legacy 'A-' format.")
    periodIndex = pd.PeriodIndex(morpcCountyEstimates["REFERENCE_PERIOD"], freq=freq.replace("Y-","A-"))
morpcCountyEstimates["DATE"] = periodIndex.to_timestamp()

Show the data.

In [ ]:
morpcCountyEstimates.head()

In [ ]:
extractedData.append(morpcCountyEstimates)

### Extract MORPC sub-county estimates

Because county estimates and sub-county estimates are generated (and regenerated) at different times, it is necessary to process each separately.

Create list of years from user-specified range.

In [ ]:
morpcEstimatesRange = list(range(MORPC_ESTIMATE_YEAR_RANGE[0], MORPC_ESTIMATE_YEAR_RANGE[1]+1))
print("Including MORPC estimates for years: {}".format(", ".join([str(x) for x in morpcEstimatesRange])))

Extract MORPC county estimates from combined table.

In [ ]:
morpcSubCountyEstimates = combinedPop.loc[
    (combinedPop["SOURCE"] == "MORPC") & 
    (combinedPop["VALUE_TYPE"] == "ESTIMATE") &
    (combinedPop["GEO_TYPE"] != "COUNTY")
].copy()

Reference period and vintage period were stored as strings, but for MORPC estimates we can treat them as integers. Convert them now.

In [ ]:
morpcSubCountyEstimates = morpcSubCountyEstimates.astype({
    "REFERENCE_PERIOD":"int",
    "VINTAGE_PERIOD":"int"
})

Verify that data is available for the specified years.

In [ ]:
if(not set(morpcEstimatesRange).issubset(set(morpcSubCountyEstimates["REFERENCE_PERIOD"]))):
    print("ERROR | Set of MORPC estimate years for which data is available does not match set derived from specified range.")
    print("Specified range: {}".format(MORPC_ESTIMATE_YEAR_RANGE))
    print("Specified set: {}".format(set(morpcEstimatesRange)))
    print("Available set: {}".format(set(morpcSubCountyEstimates["REFERENCE_PERIOD"])))
    raise RuntimeError
else:
    print("INFO | MORPC sub-county estimates data is available for all years in specified range.")

Extract only the estimates for the specified years.

In [ ]:
morpcSubCountyEstimates = morpc.extract_vintage(morpcSubCountyEstimates, refPeriods=morpcEstimatesRange)

Verify that all reference periods are present and that there is only a single vintage for each reference period.

In [ ]:
for year in morpcSubCountyEstimates["REFERENCE_PERIOD"].unique():
    temp = morpcSubCountyEstimates.loc[morpcSubCountyEstimates["REFERENCE_PERIOD"] == year]
    print("{0}: {1}".format(year, ",".join(temp["VINTAGE_PERIOD"].unique().astype("str"))))

Construct DATE field from reference period and reference period frequency.

In [ ]:
if(morpcSubCountyEstimates["REFERENCE_PERIOD_FREQ"].unique().shape[0] == 1):
    freq = morpcSubCountyEstimates["REFERENCE_PERIOD_FREQ"].iat[0]
    print("INFO | Detected reference period frequency {}".format(freq))
else:
    print("ERROR | Multiple reference period frequencies are not supported.")
    raise RuntimeError

try:
    # Hopefully this works properly with newer versions of pandas, but it has not been tested
    periodIndex = pd.PeriodIndex(morpcSubCountyEstimates["REFERENCE_PERIOD"], freq=freq)
except:
    # This works with older versions of pandas.
    print("WARNING | Error occurred when attempting to create period index using 'Y-' format. Trying legacy 'A-' format.")
    periodIndex = pd.PeriodIndex(morpcSubCountyEstimates["REFERENCE_PERIOD"], freq=freq.replace("Y-","A-"))
morpcSubCountyEstimates["DATE"] = periodIndex.to_timestamp()

Show the data.

In [ ]:
morpcSubCountyEstimates.head()

In [ ]:
extractedData.append(morpcSubCountyEstimates)

### Extract MORPC forecasts

Create list of years from user-specified range.

In [ ]:
morpcForecastsRange = list(range(MORPC_FORECAST_YEAR_RANGE[0], MORPC_FORECAST_YEAR_RANGE[1]+1, MORPC_FORECAST_YEAR_INTERVAL))
print("Including MORPC forecasts for years: {}".format(", ".join([str(x) for x in morpcForecastsRange])))

Extract MORPC forecasts from combined table.

In [ ]:
morpcForecasts = combinedPop.loc[(combinedPop["SOURCE"] == "MORPC") & (combinedPop["VALUE_TYPE"] == "FORECAST")].copy()

Reference period and vintage period were stored as strings, but for MORPC forecasts we can treat them as integers. Convert them now.

In [ ]:
morpcForecasts = morpcForecasts.astype({
    "REFERENCE_PERIOD":"int",
    "VINTAGE_PERIOD":"int"
})

Verify that data is available for the specified years.

In [ ]:
if(not set(morpcForecastsRange).issubset(set(morpcForecasts["REFERENCE_PERIOD"]))):
    print("ERROR | Set of MORPC forecast years for which data is available does not match set derived from specified range.")
    print("Specified range: {}".format(MORPC_FORECAST_YEAR_RANGE))
    print("Specified set: {}".format(set(morpcForecastsRange)))
    print("Available set: {}".format(set(morpcForecasts["REFERENCE_PERIOD"])))
    raise RuntimeError
else:
    print("INFO | MORPC forecasts data is available for all years in specified range.")

Extract only the estimates for the specified years.  The forecasts for different geographies come from different sources, so it is necessary to extract the latest vintage separately for each geography type.

In [ ]:
extractIndex = []
for group in morpcForecasts["GEO_TYPE"].unique():
    temp = morpcForecasts.loc[morpcForecasts["GEO_TYPE"] == group].copy()
    temp = morpc.extract_vintage(temp, refPeriods=morpcForecastsRange)
    extractIndex += list(temp.index)
morpcForecasts = morpcForecasts.loc[extractIndex].copy()

For each geography type, verify that all reference periods are present and that there is only a single vintage for each reference period.

In [ ]:
for group in morpcForecasts["GEO_TYPE"].unique():
    print(group)
    for year in morpcForecasts["REFERENCE_PERIOD"].unique():
        temp = morpcForecasts.loc[(morpcForecasts["REFERENCE_PERIOD"] == year) & (morpcForecasts["GEO_TYPE"] == group)]
        print("{0}: {1}".format(year, ",".join(temp["VINTAGE_PERIOD"].unique().astype("str"))))

Construct DATE field from reference period and reference period frequency.

In [ ]:
if(morpcForecasts["REFERENCE_PERIOD_FREQ"].unique().shape[0] == 1):
    freq = morpcForecasts["REFERENCE_PERIOD_FREQ"].iat[0]
    print("INFO | Detected reference period frequency {}".format(freq))
else:
    print("ERROR | Multiple reference period frequencies are not supported.")
    raise RuntimeError

try:
    # Hopefully this works properly with newer versions of pandas, but it has not been tested
    periodIndex = pd.PeriodIndex(morpcForecasts["REFERENCE_PERIOD"]+1, freq=freq)
except:
    # This works with older versions of pandas. Note the +1 offset. For whatever reason, pandas converts 2000 Y-JUN (for example)
    # to a timestamp of 1999-07-01 so we add one year to the reference period so the timestamp becomes 2000-07-01.
    print("WARNING | Error occurred when attempting to create period index using 'Y-' format. Trying legacy 'A-' format.")
    periodIndex = pd.PeriodIndex(morpcForecasts["REFERENCE_PERIOD"]+1, freq=freq.replace("Y-","A-"))
morpcForecasts["DATE"] = periodIndex.to_timestamp()

Show the data.

In [ ]:
morpcForecasts.head()

In [ ]:
extractedData.append(morpcForecasts)

### Combine extracted data

In [ ]:
combinedData = pd.concat(extractedData, axis="index")

In [ ]:
combinedData

### Compute totals for 15-county region

In [ ]:
regionTotals = combinedData.copy()
regionTotals["SUMLEVEL"] = regionTotals["GEOIDFQ"].apply(lambda x:x[0:3])
regionTotals = regionTotals.loc[regionTotals["SUMLEVEL"] == morpc.SUMLEVEL_LOOKUP["COUNTY"]].copy().drop(columns="SUMLEVEL")
regionTotals = regionTotals.drop(columns=["GEOIDFQ","GEO_TYPE","SOURCE","CONF_LEVEL"]).groupby(["REFERENCE_PERIOD","DATE"]).agg({
    "POP":"sum",
    "REFERENCE_PERIOD_FREQ":"first",
    "VINTAGE_PERIOD":"first",
    "VINTAGE_PERIOD_FREQ":"first",
    "VALUE_TYPE":"first",
    "CONF_LIMIT_UPPER":"sum",
    "CONF_LIMIT_LOWER":"sum"
}).reset_index()
regionTotals["GEOIDFQ"] = "M010000US001"
regionTotals["GEO_TYPE"] = morpc.HIERARCHY_STRING_LOOKUP["M01"]
regionTotals["SOURCE"] = "MORPC"
regionTotals.loc[regionTotals["VALUE_TYPE"] == "ESTIMATE", ["CONF_LIMIT_UPPER","CONF_LIMIT_LOWER"]] = None
regionTotals["CONF_LEVEL"] = None
regionTotals["CONF_LEVEL"] = regionTotals["CONF_LEVEL"].astype("float")
regionTotals.head()

In [ ]:
combinedData = pd.concat([combinedData, regionTotals], axis="index")

### Reformat combined data for output

Create a working dataframe to prepare for export.

In [ ]:
insightsPop = combinedData.copy()

Merge the geography name, sumlevel, and county FIPS code from the geography lookup table, aligning on fully-qualified GEOID.

In [ ]:
insightsPop = insightsPop.merge(geoLookup.reset_index()[["GEOIDFQ","NAME","SUMLEVEL","COUNTYFP"]], on="GEOIDFQ")

Add a suffix to township names that includes the word "township" and the name of the county where they are located.  This is necessary to differentiate them from places and to disambiguate townships having the same name in different counties.

In [ ]:
townshipsTemp = insightsPop.loc[insightsPop["SUMLEVEL"] == morpc.SUMLEVEL_LOOKUP["COUNTY-TOWNSHIP-REMAINDER"]].copy()
townshipsTemp["COUNTYID"] = "39"+townshipsTemp["COUNTYFP"]
townshipsTemp["COUNTY"] = townshipsTemp["COUNTYID"].map(morpc.CONST_COUNTY_ID_TO_NAME)
townshipsTemp["NAME"] = townshipsTemp["NAME"] + " Township (" + townshipsTemp["COUNTY"] + ")" 
insightsPop.update(townshipsTemp, overwrite=True)

Add a suffix to county names that includes the word "County". This is necessary to differentiate them from places and townships that have the same name.

In [ ]:
countyTemp = insightsPop.loc[insightsPop["SUMLEVEL"] == morpc.SUMLEVEL_LOOKUP["COUNTY"]].copy()
countyTemp["NAME"] = countyTemp["NAME"] + " County"
insightsPop.update(countyTemp, overwrite=True)

Extract only the required fields.

In [ ]:
insightsPop = insightsPop.filter(items=["POP","GEOIDFQ","GEO_TYPE","NAME","DATE","VALUE_TYPE","CONF_LIMIT_UPPER","CONF_LIMIT_LOWER","SOURCE","VINTAGE_PERIOD"], axis="columns")

Split estimates and forecasts into separate columns.

In [ ]:
insightsPop = insightsPop.pivot(index=["GEOIDFQ","GEO_TYPE","NAME","DATE","CONF_LIMIT_UPPER","CONF_LIMIT_LOWER","SOURCE","VINTAGE_PERIOD"], columns="VALUE_TYPE", values="POP").reset_index()

Give the columns more human-friendly names.

In [ ]:
insightsPop = insightsPop.rename(columns={
    "NAME":"Name",
    "GEO_TYPE":"Geography type",
    "DATE":"Date",
    "ESTIMATE":"Historical",
    "FORECAST":"Forecast",
    "CONF_LIMIT_UPPER":"Confidence limit (upper)",
    "CONF_LIMIT_LOWER":"Confidence limit (lower)",
    "SOURCE":"Source",
    "VINTAGE_PERIOD":"Vintage year"
})

Extract only the fields listed in the schema (should just be a precaution).

In [ ]:
insightsPop = insightsPop.filter(items=insightsPopSchema.field_names, axis="columns")

Make sure the fields all have the data type expected by the schema.

In [ ]:
insightsPop = morpc.cast_field_types(insightsPop, insightsPopSchema)

Sort the values by geography type, then name, then date.

In [ ]:
insightsPop = insightsPop.sort_values(["Geography type","Name","Date"])

Inspect the data.

In [ ]:
insightsPop.head()

### Suppress select data

#### Suppress forecasts for small communties

Suppress forecasts for communities whose historical population is below the threshold defined in FORECAST_POP_THRESHOLD.  Actually, the intent here is to exclude communities that have small geographies because of the potential for error when reapportioning GridMAZ values.  This would probably benefit from a more rigorous method.

In [ ]:
maxPop = insightsPop[["GEOIDFQ","Historical"]].groupby("GEOIDFQ").max().max(axis="columns")
noForecastGeoids = maxPop.loc[maxPop < FORECAST_POP_THRESHOLD].index
noForecastIndex = insightsPop.loc[insightsPop["GEOIDFQ"].isin(noForecastGeoids)].index
print("Forecasts will not be included for {} communities whose historical population is below the threshold ({}).".format(len(noForecastGeoids), FORECAST_POP_THRESHOLD))

Show the largest communities whose forecasts will not be included.

In [ ]:
temp = insightsPop.groupby("GEOIDFQ").agg({
    "Name":"first",
    "Geography type":"first",
    "Historical":"max"
}).reset_index()
temp.loc[temp["GEOIDFQ"].isin(noForecastGeoids)].sort_values("Historical", ascending=False).head(10)

Suppress the data.

In [ ]:
insightsPop = insightsPop.drop(index=noForecastIndex)

#### Suppress all data for very low population communities

Suppress all data for communities smaller than the threshold defined in INCLUSION_POP_THRESHOLD.  This will achieve two things: it will eliminate communities whose estimates may embody a large margin of error and it will eliminate communities on the border of the region whose population mostly live outside of the region.  This would probably benefit from a more rigorous method.

In [ ]:
minPop = insightsPop[["GEOIDFQ","Historical","Forecast"]].groupby("GEOIDFQ").min().min(axis="columns")
excludeGeoids = minPop.loc[minPop < INCLUSION_POP_THRESHOLD].index
excludeIndex = insightsPop.loc[insightsPop["GEOIDFQ"].isin(excludeGeoids)].index
print("No data will be included for {} communities whose historical population is below the threshold ({})".format(len(excludeGeoids), INCLUSION_POP_THRESHOLD))

Show the largest communities which will not be included. Note that the maximum population is shown and that this can be greater than the threshold. The minimum historical population was used for the comparison.  When large discrepancies are apparent, this is typically because the place is on the edge of the region (e.g. Galion). In this case, the census estimates account for the entire place and the MORPC estimate accounts for only the portion within the region.  Ideally we would fix this in upstream processes.

In [ ]:
temp = insightsPop.groupby("GEOIDFQ").agg({
    "Name":"first",
    "Geography type":"first",
    "Historical":"max"
}).reset_index()
temp.loc[temp["GEOIDFQ"].isin(excludeGeoids)].sort_values("Historical", ascending=False).head(10)

Suppress the data.

In [ ]:
insightsPop = insightsPop.drop(index=excludeIndex)

## Export data

In [ ]:
insightsPop.to_csv(INSIGHTS_POP_TABLE_PATH, index=False)

## Create resource file for exported data

In [ ]:
insightsPopResource = morpc.frictionless_create_resource(INSIGHTS_POP_TABLE_FILENAME, 
    resourcePath=INSIGHTS_POP_TABLE_RESOURCE_PATH,
    title="MORPC Insights | Historic and Forecasted Population by Year", 
    name="morpc_insights_pop_temporal", 
    description="This dataset provides the best available historical and forecasted population estimates for the Central Ohio region and the counties and communities therein.  Estimates are compiled from a variety of sources including the following: {}.  Note that different sources provide estimates as of different days throughout the year, so be sure to note the entire date in the REFERENCE_PERIOD field.  The VINTAGE_PERIOD field refers to the time that the estimate was produced or released.  Earlier and (perhaps) later vintages may be available from the original sources.".format(", ".join(["{1} ({0})".format(value, key) for key, value in SOURCE_MAP.items()])),
    writeResource=True,
    validate=True
)

## Generate static charts

The following block produces a set of static charts (i.e. thumbnail images) in SVG format.  A chart is produced for each MORPC member geography, except those which might have been suppressed previously and communities for which there is only a single data point (i.e. townships outside of the modeling area).  See inline comments below.  Because some geographies could be excluded, the block produces a list of those that are included so that they can also be included in the catalog generated in the next section.

In [ ]:
%matplotlib agg

for f in os.scandir(chartDir):
    os.remove(f)

# Create a list to accumulate geographies for which a thumbnail is generated
platformIncludeList = []
# Iterate over each geography in data set
for geoid in insightsPop["GEOIDFQ"].unique():
    # If the geography is not a MORPC member, skip it. The platform only features members.
    if(not geoid in memberList):
        continue
    # Extract the data for a single geography
    temp = insightsPop.loc[insightsPop["GEOIDFQ"] == geoid].copy()
    # Determine the maximum population that appears for this geography. We will use this
    # to determine the y-axis limit
    maxPop = temp[["Historical","Forecast","Confidence limit (upper)"]].max().max()

    # The following lines can be used to include only five year intervals in the historic data, plus the most
    # recent MORPC estimate. As of Feb 2025, we decided to include all historic values so this is disabled.
    #temp['year'] = temp["Date"].dt.year
    #temp = temp.loc[(temp['year'] % 5 == 0) | (temp['year'].isin(range(MORPC_ESTIMATE_YEAR_RANGE[0], MORPC_ESTIMATE_YEAR_RANGE[1]+1)))]

    # The following lines were anticipated to be used to display the forecast vintage in the legend entry of the forecast, 
    # values, however as of Feb 2025 we decided not to do this. The case forecastVintage=None is used, however, to skip
    # plotting when no forecast values are available
    forecastVintages = list(temp.loc[temp["Forecast"].notna(), "Vintage year"].unique())
    if(len(forecastVintages) > 1):
        forecastVintage = "multiple"
    elif(len(forecastVintages) == 0):
        forecastVintage = None
    else: 
        forecastVintage = forecastVintages[0]

    # If the data contains only one data point, skip it.  This usually occurs for townships outside of the modeling area which
    # do not have historic census data and do not have forecast data.
    if(temp.shape[0] == 1):
        continue

    # If we made it this far the geo will not be excluded. Add it to the list.
    platformIncludeList.append(geoid)

    # Generate a title string
    geoName = temp.iloc[0]["Name"]
    geoType = temp.iloc[0]["Geography type"]
    geoLabel = "Population - {}{}".format(geoName, GEO_TYPE_LABELS[geoType]) 

    # Create and annotate the plot
    PLOTWIDTH = 8
    fig,ax = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))
    if(not temp["Confidence limit (upper)"].isnull().all()):
        temp.plot(ax=ax, x="Date", y="Confidence limit (upper)", color="#BFBFBF", linewidth=2, label="Confidence limit")
        temp.plot(ax=ax, x="Date", y="Confidence limit (lower)", color="#BFBFBF", linewidth=2, legend=False)
    temp.plot(ax=ax, x="Date", y="Historical", marker="o", color=morpc.CONST_MORPC_COLORS["darkblue"], linewidth=3, markersize=7, label="Historical best estimate")
    if(forecastVintage != None):
        temp.plot(ax=ax, x="Date", y="Forecast", marker="o", color=morpc.CONST_MORPC_COLORS["darkgreen"], linewidth=3, markersize=7, label="Forecast")
    ax.set_title(geoLabel, fontsize=14)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.grid(visible=True, color="lightgrey")
    # Set the y-limit to 10% more than the max pop value
    ax.set_ylim(ymin=0, ymax=maxPop*1.1)   
    # Format the y-axis labels as integers with comma separators
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    # Save the figure to disk as an SVG file
    ax.figure.savefig(os.path.join(chartDir, "{}.svg".format(geoid)))
    
    plt.close(ax.figure)

    # Extract only the columns required to generate a plot in Excel. Put the confidence limit columns
    # before the forecast column so that their lines appear under the forecast line
    excelData = temp[["Date","Historical","Confidence limit (upper)","Confidence limit (lower)","Forecast",]].copy()

    if(excelData["Confidence limit (upper)"].isnull().all()):
        excelData = excelData.drop(columns=["Confidence limit (upper)","Confidence limit (lower)"])
    if(excelData["Forecast"].isnull().all()):
        excelData = excelData.drop(columns=["Forecast"])

    dateFormat = "yyyy-mm-dd"
    writer = pd.ExcelWriter(os.path.join(chartDir, "{}.xlsx".format(geoid)), engine='xlsxwriter', datetime_format=dateFormat)
    dataOptions = {
        "numberFormat": {
            # Because the Date field is the index, the format here is overridden by pd.ExcelWriter's datetime_format
            # argument. It is included here to keep morpc.data_chart_to_excel from complaining
            "Date": dateFormat, 
            "Historical": "#,##0",
            "Forecast": "#,##0",
            "Confidence limit (upper)": "#,##0",
            "Confidence limit (lower)": "#,##0"
        },
        "columnWidth": 20
    }
    chartOptions = {
        "colors": {
            "Historical": morpc.CONST_MORPC_COLORS["darkblue"],
            "Forecast": morpc.CONST_MORPC_COLORS["darkgreen"],
            "Confidence limit (upper)": "#BFBFBF",
            "Confidence limit (lower)": "#BFBFBF"
        },            
        "titles": {
            "chartTitle": geoLabel,
            "xTitle": None,
            "yTitle": None
        },
        "seriesOptions": {
            "Historical": {
                "line": {"width":3},
                "marker": {"type":"circle", "size":7}
            },
            "Forecast": {
                "line": {"width":3},
                "marker": {"type":"circle", "size":7}
            },
            "Confidence limit (upper)": {
                "line": {"width":3},
                "marker": None
            },            
            "Confidence limit (lower)": {
                "line": {"width":3},
                "marker": None
            },
        },
        "xAxisOptions": {
            'date_axis':  True,
            'num_format': 'yyyy',
            'major_unit': 10,
            'major_unit_type': "years",
            "num_font": {"size":16}
        },
        "yAxisOptions": {
            "min": 0, 
            "max": 1.1*maxPop,
            "num_font": {"size":16},
            "num_format": '[>=1000]#,##0,"K"',
        },
        "sizeOptions": {
            "x_scale": 1.5,
            "y_scale": 1.5
        },
        "plotAreaLayout": {
            'x': 0.14,
            'y': 0.13,
            'width': 0.8,
            'height': 0.67
        }
    }
    morpc.data_chart_to_excel(excelData.set_index("Date"), writer, chartType="line", dataOptions=dataOptions, chartOptions=chartOptions)
    writer.close()

%matplotlib inline

## Generate Insights catalog content

The content in the Insights platform is controlled by a catalog spreadsheet. Each tile to be displayed in the platform must have a record in the catalog.  This section will create the records for the tiles that display the temporal pop data.  Eventually this function will be performed by a separate staging script.

First specify the column names used in the catalog.

In [ ]:
columnNames=["TileID","TilesetID","GeographyType","GeographyName","Category","Headline","Commentary","ThumbnailURL","Contributor","Vintage","UpdateInterval","ShareURL","DataProductURL","MoreInformationURL"]

Create a new dataframe containing only the geographies for which thumbnail images were produced in the section above.

In [ ]:
catalog = insightsPop.loc[insightsPop["GEOIDFQ"].isin(platformIncludeList)].copy()

Extract only the metadata columns of interest and flatten the data to have only one record per geography. Rename the metadata fields to match the catalog fields.

In [ ]:
catalog = catalog.filter(items=["GEOIDFQ","Name","Geography type"], axis="columns") \
    .groupby("GEOIDFQ").first() \
    .reset_index() \
    .rename(columns={"Name":"GeographyName","Geography type":"GeographyType"})

Change the GeographyType values to match the schema of the catalog.

In [ ]:
catalog["GeographyType"] = catalog["GeographyType"].map({
    "REGION15":"Region",
    "COUNTY":"County",
    "COUNTY-TOWNSHIP-REMAINDER":"Community",
    "PLACE":"Community"
})

Populate some placeholder fields.

In [ ]:
catalog["TileID"] = None
catalog["TilesetID"] = None
catalog["Category"] = None
catalog["Headline"] = "TBD"
catalog["Commentary"] = "TBD"

Generate the URL for the thumbnail images. These will be hosted in GitHub and will be indexed by GEOIDFQ.

In [ ]:
catalog["ThumbnailURL"] = catalog["GEOIDFQ"].apply(lambda geoid:"https://raw.githubusercontent.com/morpc-insights/pop-temporal/refs/heads/main/output_data/charts/{}.svg".format(geoid))

Populate some other simple metadata.  Vintage in this case refers to the year that the content was published in Insights. This is to give readers an idea of how old it is.  UpdateInterval gives them an idea of when to expect the next version. ShareURL is a placeholder for now.

In [ ]:
catalog["Contributor"] = "Mid-Ohio Regional Planning Commission"
catalog["Vintage"] = str(datetime.date.today().year)
catalog["UpdateInterval"] = "annually"
catalog["ShareURL"] = None

Generate the data product URL.  This points to an ArcGIS Dashboard that accepts URL parameters.  GEOIDFQ is passed as a parameter to tell the app to load the data for a particular geography.

In [ ]:
catalog["DataProductURL"] = catalog["GEOIDFQ"].apply(lambda geoid:"https://morpc.maps.arcgis.com/apps/dashboards/d1291225631545438293bdfcfffaef6b#region={}".format(geoid))

Generate the URLs that point to the extended commentary pages.  Default to a common page (population.pdf) hosted in GitHub.  Point to specific pages for the 15-county region and for each county.

In [ ]:
temp["MoreInformationURL"] = "https://raw.githubusercontent.com/morpc-insights/pop-temporal/refs/heads/main/fact_sheets/population.pdf"
temp = catalog.loc[catalog["GeographyType"].isin(["COUNTY","REGION15"])].copy()
temp["MoreInformationURL"] = temp["GEOIDFQ"].apply(lambda geoid:"https://raw.githubusercontent.com/morpc-insights/pop-temporal/refs/heads/main/fact_sheets/{}.pdf".format(geoid))
catalog.update(temp)

Extract only the required columns.

In [ ]:
catalog = catalog.filter(items=columnNames, axis="columns")

Inspect the listing.

In [ ]:
catalog.head()

Save the catalog to an Excel spreadsheet.

In [ ]:
catalog.to_excel("catalog.xlsx", index=False)

It is necessary to manually add these records to the master catalog or update the records already therein.  See the following file in GitHub:

https://github.com/morpc/morpc-insights/blob/main/catalog/morpc_insights_catalog.xlsx
